## 1. Web Scraping the School Districts in Texas

In [1]:
import requests
import re
from helper_fxns import *
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
url = "https://ballotpedia.org/List_of_school_districts_in_Texas"

In [3]:
response = requests.get(url)
response.status_code

200

In [4]:
page = response.text
soup = BeautifulSoup(page)

In [5]:
lst_of_links = list(soup.find_all('a'))
lst_of_links[0:5]

[<a class="btn-block btn-lg btn-primary" href="https://www.facebook.com/sharer/sharer.php?u=https%3A%2F%2Fballotpedia.org%2FList_of_school_districts_in_Texas" target="_blank"><span class="fa fa-facebook"></span>   Share this page</a>,
 <a class="btn-block btn-lg btn-primary" href="https://www.facebook.com/Ballotpedia/" target="_blank"><span class="fa fa-facebook"></span>   Follow Ballotpedia</a>,
 <a class="btn-block btn-lg btn-info" href="https://twitter.com/home?status=https%3A%2F%2Fballotpedia.org%2FList_of_school_districts_in_Texas" target="_blank"><span class="fa fa-twitter"></span>   Share this page</a>,
 <a class="btn-block btn-lg btn-info" href="https://twitter.com/Ballotpedia/" target="_blank"><span class="fa fa-twitter"></span>   Follow Ballotpedia</a>,
 <a data-target="#modal-email" data-toggle="modal" href="#" id="bp-header-email-toggle"><span class="glyphicon glyphicon-envelope"></span><span class="hide-mobile-2"> Subscribe</span></a>]

In [6]:
#compile regex to find all instances of strings with "Texas"
texas_regex = re.compile('Texas')
tx_strs = soup.find_all(text=texas_regex)

In [7]:
#get first and last indices to slice list
#remove texas and commas and CISD, ISD, and CSD from the strings
start_index = tx_strs.index('Abbott Independent School District, Texas')
end_index = tx_strs.index('Zephyr Independent School District, Texas')
tx_strs = tx_strs[start_index : end_index + 1]
for i,district in enumerate(tx_strs):
    tx_strs[i] = tx_strs[i].replace(',','').replace(' Texas', '').\
    replace(' Independent School District','').replace(' Consolidated','').\
    replace(' Common School District', '')
tx_strs[0:5]

['Abbott', 'Abernathy', 'Abilene', 'Academy', 'Adrian']

In [8]:
len(tx_strs)

1010

## 2. Creating School District DataFrame

In [9]:
sd_df = pd.DataFrame({'School District':tx_strs})
sd_df.head()

#use cols when reading in csvs to get specific cols
#to use in visual studio #%% to create cells and then type awau

,School District
0,Abbott
1,Abernathy
2,Abilene
3,Academy
4,Adrian


## Merging CSVs into DataFrame

In [10]:
#2016 SAT data
#sat_df = pd.read_csv('./sat_train_val_data_2016.csv')
#sat_df

In [11]:
sat_df = pd.read_csv('./sat_district_data_class_2016.csv',header= 0, names=['Group','District Number','District', 'County', '2016 class SAT mean'],\
                     usecols = [0,1,2,4,10])


In [12]:
#filter for Group == to All Students to remove subsets of students
is_all_students = sat_df['Group'] =='All Students'
sat_df = sat_df[is_all_students]#.loc[:,:]
sat_df['District Number'] = sat_df['District Number'].str.replace('=','').str.replace('"', '').astype(int)
sat_df.head()

,Group,District Number,District,County,2016 class SAT mean
0,All Students,1902,Cayuga ISD,Anderson County,978.0
42,All Students,1903,Elkhart ISD,Anderson County,978.0
84,All Students,1904,Frankston ISD,Anderson County,932.0
126,All Students,1906,Neches ISD,Anderson County,971.0
168,All Students,1907,Palestine ISD,Anderson County,918.0


In [13]:
#drop Group col
sat_df.drop('Group', axis =1, inplace= True)
sat_df.head()

,District Number,District,County,2016 class SAT mean
0,1902,Cayuga ISD,Anderson County,978.0
42,1903,Elkhart ISD,Anderson County,978.0
84,1904,Frankston ISD,Anderson County,932.0
126,1906,Neches ISD,Anderson County,971.0
168,1907,Palestine ISD,Anderson County,918.0


In [14]:
#filter for only public school district aka ISD or CISD
#is_public_school_dis = sat_df['District'].str.match('.*ISD')
#sat_df = sat_df[is_public_school_dis]
sat_df = get_public_school_dis(sat_df, 'District')
sat_df.head()

,District Number,District,County,2016 class SAT mean
0,1902,Cayuga ISD,Anderson County,978.0
42,1903,Elkhart ISD,Anderson County,978.0
84,1904,Frankston ISD,Anderson County,932.0
126,1906,Neches ISD,Anderson County,971.0
168,1907,Palestine ISD,Anderson County,918.0


In [15]:
#remove CISD or ISD from District names in District col
#is_isd_cisd = re.compile(r'\s(CISD|ISD)')
#sat_df['District'] =sat_df['District'].replace(to_replace=is_isd_cisd, value='')                                              
sat_df = remove_cisd_isd_from_col(sat_df, 'District')
sat_df.head()
#sat_df['District'] = sat_df['District']

,District Number,District,County,2016 class SAT mean
0,1902,Cayuga,Anderson County,978.0
42,1903,Elkhart,Anderson County,978.0
84,1904,Frankston,Anderson County,932.0
126,1906,Neches,Anderson County,971.0
168,1907,Palestine,Anderson County,918.0


In [16]:
#filter for rows with non null SAT vals
sat_df.sort_values(by=['District'], inplace=True)
not_is_nan_sat = sat_df['2016 class SAT mean'].notna()
sat_df = sat_df[not_is_nan_sat] #.loc[:,:]
sat_df.head()

,District Number,District,County,2016 class SAT mean
19488,109901,Abbott,Hill County,1054.0
15750,95901,Abernathy,Hale County,1100.0
35154,221901,Abilene,Taylor County,948.0
1722,14901,Academy,Bell County,1068.0
2646,15901,Alamo Heights,Bexar County,1083.0


In [17]:
#join the sat df with the school district names df
sat_df = sat_df.set_index('District').join(sd_df.set_index('School District'))
sat_df.head()

,District Number,County,2016 class SAT mean
Abbott,109901,Hill County,1054.0
Abernathy,95901,Hale County,1100.0
Abilene,221901,Taylor County,948.0
Academy,14901,Bell County,1068.0
Alamo Heights,15901,Bexar County,1083.0


In [18]:
#remove any nulls
sat_df.dropna(axis=0, how='any', inplace= True)
sat_df.reset_index(inplace=True)
sat_df = sat_df.rename(columns={'index': 'School District'})
sat_df.head()


,School District,District Number,County,2016 class SAT mean
0,Abbott,109901,Hill County,1054.0
1,Abernathy,95901,Hale County,1100.0
2,Abilene,221901,Taylor County,948.0
3,Academy,14901,Bell County,1068.0
4,Alamo Heights,15901,Bexar County,1083.0


In [19]:
#get rid of duplicates
sat_df = sat_df.groupby(['District Number','School District','County']).agg('mean')
sat_df.reset_index(inplace=True)
sat_df.head()

,District Number,School District,County,2016 class SAT mean
0,1902,Cayuga,Anderson County,978.0
1,1903,Elkhart,Anderson County,978.0
2,1904,Frankston,Anderson County,932.0
3,1906,Neches,Anderson County,971.0
4,1907,Palestine,Anderson County,918.0


In [20]:
#join 2014-2015 funding excel into sat_df
funds_df = pd.read_excel('./Total Operating Expenditures 2006-2017.xlsx', header = 1,names=['District Number',\
            'School District', 'Expenditures per Student'],usecols = [0,1,31], nrows=1308)
funds_df.head()

,District Number,School District,Expenditures per Student
0,1902,CAYUGA ISD,14278.0
1,1903,ELKHART ISD,8509.0
2,1904,FRANKSTON ISD,8994.0
3,1906,NECHES ISD,10349.0
4,1907,PALESTINE ISD,10021.0


In [21]:
#filter for only public schools and remove CISD or ISD from names
funds_df = get_public_school_dis(funds_df, 'School District')
remove_cisd_isd_from_col(funds_df, 'School District')
funds_df.head()

,District Number,School District,Expenditures per Student
0,1902,CAYUGA,14278.0
1,1903,ELKHART,8509.0
2,1904,FRANKSTON,8994.0
3,1906,NECHES,10349.0
4,1907,PALESTINE,10021.0


In [22]:
#check for duplicates
print(sum(funds_df.duplicated(keep=False) == True))

#Modify School District strings to first characters of all the words capitalized
funds_df['School District'] = funds_df['School District'].str.title()
funds_df.head()

0


,District Number,School District,Expenditures per Student
0,1902,Cayuga,14278.0
1,1903,Elkhart,8509.0
2,1904,Frankston,8994.0
3,1906,Neches,10349.0
4,1907,Palestine,10021.0


In [23]:
#join funds_df with the sat_df
#sat_df = sat_df.set_index('School District').join(funds_df.set_index('School District') on='School District', 'District Number'
sat_df = sat_df.merge(funds_df, on=('School District', 'District Number'))
sat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 794 entries, 0 to 793
Data columns (total 5 columns):
District Number             794 non-null int64
School District             794 non-null object
County                      794 non-null object
2016 class SAT mean         794 non-null float64
Expenditures per Student    794 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 37.2+ KB


In [24]:
#sat_df.reset_index(inplace=True)
sat_df.head()

,District Number,School District,County,2016 class SAT mean,Expenditures per Student
0,1902,Cayuga,Anderson County,978.0,14278.0
1,1903,Elkhart,Anderson County,978.0,8509.0
2,1904,Frankston,Anderson County,932.0,8994.0
3,1906,Neches,Anderson County,971.0,10349.0
4,1907,Palestine,Anderson County,918.0,10021.0


In [25]:
null_columns=sat_df.columns[sat_df.isnull().any()]
print(sat_df[sat_df.isnull().any(axis=1)][null_columns].head())

Empty DataFrame
Columns: []
Index: []


In [26]:
#get rid of duplicates by averaging them and reset the index
#sat_df = sat_df.groupby(['School District','County', '2016 class SAT mean']).agg('mean')


In [27]:
#remove nulls and reset index

#sat_df.dropna(axis=0, how='any', inplace= True)
#sat_df.reset_index(inplace=True)

In [28]:
print(sat_df.head())
sat_df.info()

   District Number School District           County  2016 class SAT mean  \
0             1902          Cayuga  Anderson County                978.0   
1             1903         Elkhart  Anderson County                978.0   
2             1904       Frankston  Anderson County                932.0   
3             1906          Neches  Anderson County                971.0   
4             1907       Palestine  Anderson County                918.0   

   Expenditures per Student  
0                   14278.0  
1                    8509.0  
2                    8994.0  
3                   10349.0  
4                   10021.0  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 794 entries, 0 to 793
Data columns (total 5 columns):
District Number             794 non-null int64
School District             794 non-null object
County                      794 non-null object
2016 class SAT mean         794 non-null float64
Expenditures per Student    794 non-null float64
dtypes: float64(2)

In [29]:
#join 2014-2015 economically disadvantaged count csv into sat_df
econ_dis_df = pd.read_csv('./Economically Disadvantaged Report_Statewide_Districts_2014-2015.csv', header = 2,\
                            usecols = [2,3,5,7,9]) #names=['School District', 'Not Economically Disadvantaged Student Count']
econ_dis_df.head()

,District Name,District Number,Eligible For Free Meals Count,Eligible For Reduced-Price Meals Count,Other Economically Disadvantaged Count
0,A W BROWN-FELLOWSHIP LEADERSHIP AC,57816,1419,402,0
1,A+ ACADEMY,57829,804,110,0
2,ABBOTT ISD,109901,38,21,0
3,ABERNATHY ISD,95901,343,76,0
4,ABILENE ISD,221901,9228,1692,49


In [30]:
econ_dis_df.columns = econ_dis_df.columns.str.strip()

In [31]:
#remove negative numbers from df
#remove_neg_nums = econ_dis_df['Economically Disadvantaged Student Count'] >= 0
#econ_dis_df = econ_dis_df[remove_neg_nums]
#econ_dis_df.reset_index(drop=True, inplace=True)

econ_dis_df = econ_dis_df.replace(-9999999,0)
econ_dis_df.head()

,District Name,District Number,Eligible For Free Meals Count,Eligible For Reduced-Price Meals Count,Other Economically Disadvantaged Count
0,A W BROWN-FELLOWSHIP LEADERSHIP AC,57816,1419,402,0
1,A+ ACADEMY,57829,804,110,0
2,ABBOTT ISD,109901,38,21,0
3,ABERNATHY ISD,95901,343,76,0
4,ABILENE ISD,221901,9228,1692,49


In [32]:
#get total economically disadvantaged students
econ_dis_df['Economically Disadvantaged Student Count'] = econ_dis_df['Eligible For Free Meals Count']\
+econ_dis_df['Eligible For Reduced-Price Meals Count'] + econ_dis_df['Other Economically Disadvantaged Count']

econ_dis_df.drop(columns=['Eligible For Free Meals Count','Eligible For Reduced-Price Meals Count','Other Economically Disadvantaged Count'], inplace=True)
econ_dis_df.head()

,District Name,District Number,Economically Disadvantaged Student Count
0,A W BROWN-FELLOWSHIP LEADERSHIP AC,57816,1821
1,A+ ACADEMY,57829,914
2,ABBOTT ISD,109901,59
3,ABERNATHY ISD,95901,419
4,ABILENE ISD,221901,10969


In [33]:
econ_dis_df = get_public_school_dis(econ_dis_df, 'District Name')
econ_dis_df = remove_cisd_isd_from_col(econ_dis_df, 'District Name')
econ_dis_df['District Name'] = econ_dis_df['District Name'].str.title().str.strip()
econ_dis_df.head()

,District Name,District Number,Economically Disadvantaged Student Count
2,Abbott,109901,59
3,Abernathy,95901,419
4,Abilene,221901,10969
6,Academy,14901,538
11,Adrian,180903,76


In [34]:
econ_dis_df = econ_dis_df.rename(columns={'District Name': 'School District'})
#sat_df = sat_df.set_index('School District').join(econ_dis_df.set_index('School District'))
sat_df = sat_df.merge(econ_dis_df,on=('School District','District Number'))
#sat_df.reset_index(inplace=True)
sat_df.head()

,District Number,School District,County,2016 class SAT mean,Expenditures per Student,Economically Disadvantaged Student Count
0,1902,Cayuga,Anderson County,978.0,14278.0,210
1,1903,Elkhart,Anderson County,978.0,8509.0,648
2,1904,Frankston,Anderson County,932.0,8994.0,446
3,1906,Neches,Anderson County,971.0,10349.0,156
4,1907,Palestine,Anderson County,918.0,10021.0,2458


In [35]:
#check for duplicates
#check for duplicates
#print(sum(sat_df.duplicated() == True))
#sat_df[sat_df.duplicated(subset='District Number')  == True]

In [36]:
#join 2014-2015 average daily attendance excel into sat_df
ada_df = pd.read_excel('./County ADA Report 2005-2018.xlsx', header = 0 ,names=['District Number',\
            'School District','Average Daily Attendance'],usecols = [2,3,12])
ada_df.head()

,District Number,School District,Average Daily Attendance
0,1902,CAYUGA ISD,560.125
1,1903,ELKHART ISD,1134.768
2,1904,FRANKSTON ISD,713.236
3,1906,NECHES ISD,396.754
4,1907,PALESTINE ISD,3052.135


In [37]:
ada_df = get_public_school_dis(ada_df, 'School District')
ada_df = remove_cisd_isd_from_col(ada_df, 'School District')
ada_df['School District'] = ada_df['School District'].str.title()
ada_df.head()

,District Number,School District,Average Daily Attendance
0,1902,Cayuga,560.125
1,1903,Elkhart,1134.768
2,1904,Frankston,713.236
3,1906,Neches,396.754
4,1907,Palestine,3052.135


In [38]:
sat_df = sat_df.merge(ada_df,on=('School District','District Number'))
#sat_df.drop('index', axis = 1, inplace=True)
sat_df.head()

,District Number,School District,County,2016 class SAT mean,Expenditures per Student,Economically Disadvantaged Student Count,Average Daily Attendance
0,1902,Cayuga,Anderson County,978.0,14278.0,210,560.125
1,1903,Elkhart,Anderson County,978.0,8509.0,648,1134.768
2,1904,Frankston,Anderson County,932.0,8994.0,446,713.236
3,1906,Neches,Anderson County,971.0,10349.0,156,396.754
4,1907,Palestine,Anderson County,918.0,10021.0,2458,3052.135


In [39]:
#join 2014-2015 annual dropout rate excel into sat_df
drop_df = pd.read_excel('./District_Data_Download_Drop.xlsx', sheet_name= "Drop_1415", header = 0,\
                        names=['Calc for state Accountability','Gradespan','District Number',\
                               'School District','Annual Dropout Rate'],usecols = [0,1,2,3,9])
drop_df.head()

,Calc for state Accountability,Gradespan,District Number,School District,Annual Dropout Rate
0,No,712,1902,Cayuga ISD,1.0
1,Yes,712,1902,Cayuga ISD,1.0
2,No,78,1902,Cayuga ISD,0.0
3,Yes,78,1902,Cayuga ISD,0.0
4,No,912,1902,Cayuga ISD,1.4


In [40]:
#Filter for 7-12 grades and Calculated for State Accountability equal to yes
grade_712 = drop_df['Gradespan'] == 712
state_acct = drop_df['Calc for state Accountability'] == 'Yes'
drop_df = drop_df[grade_712 & state_acct]
drop_df.head()

,Calc for state Accountability,Gradespan,District Number,School District,Annual Dropout Rate
1,Yes,712,1902,Cayuga ISD,1.0
7,Yes,712,1903,Elkhart ISD,0.0
13,Yes,712,1904,Frankston ISD,0.3
19,Yes,712,1906,Neches ISD,1.0
25,Yes,712,1907,Palestine ISD,0.6


In [41]:
drop_df = get_public_school_dis(drop_df, 'School District')
drop_df = remove_cisd_isd_from_col(drop_df, 'School District')
drop_df.head()

,Calc for state Accountability,Gradespan,District Number,School District,Annual Dropout Rate
1,Yes,712,1902,Cayuga,1.0
7,Yes,712,1903,Elkhart,0.0
13,Yes,712,1904,Frankston,0.3
19,Yes,712,1906,Neches,1.0
25,Yes,712,1907,Palestine,0.6


In [42]:
drop_df.drop(columns=['Calc for state Accountability', 'Gradespan'], inplace= True)
drop_df.head()

,District Number,School District,Annual Dropout Rate
1,1902,Cayuga,1.0
7,1903,Elkhart,0.0
13,1904,Frankston,0.3
19,1906,Neches,1.0
25,1907,Palestine,0.6


In [43]:
sat_df = sat_df.merge(drop_df,on=('School District','District Number'))
#sat_df.drop('index', axis = 1, inplace=True)
sat_df.head()

,District Number,School District,County,2016 class SAT mean,Expenditures per Student,Economically Disadvantaged Student Count,Average Daily Attendance,Annual Dropout Rate
0,1902,Cayuga,Anderson County,978.0,14278.0,210,560.125,1.0
1,1903,Elkhart,Anderson County,978.0,8509.0,648,1134.768,0.0
2,1904,Frankston,Anderson County,932.0,8994.0,446,713.236,0.3
3,1906,Neches,Anderson County,971.0,10349.0,156,396.754,1.0
4,1907,Palestine,Anderson County,918.0,10021.0,2458,3052.135,0.6


In [44]:
sat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 0 to 792
Data columns (total 8 columns):
District Number                             793 non-null int64
School District                             793 non-null object
County                                      793 non-null object
2016 class SAT mean                         793 non-null float64
Expenditures per Student                    793 non-null float64
Economically Disadvantaged Student Count    793 non-null int64
Average Daily Attendance                    793 non-null float64
Annual Dropout Rate                         793 non-null float64
dtypes: float64(4), int64(2), object(2)
memory usage: 55.8+ KB


In [45]:
#sat_df[sat_df['School District'].duplicated()]

In [46]:
#join LEAID into sat_df in order to link data from nces with data in sat_df

In [47]:
leaid_df = pd.read_excel('./EDGE_GEOCODE_PUBLICLEA_1718.xlsx', header = 0,\
                        names=['LEAID','School District','State','County'],usecols = [0,1,5,9])
leaid_df.head()

,LEAID,School District,State,County
0,100002,Alabama Youth Services,AL,Jefferson County
1,100005,Albertville City,AL,Marshall County
2,100006,Marshall County,AL,Marshall County
3,100007,Hoover City,AL,Jefferson County
4,100008,Madison City,AL,Madison County


In [48]:
tx_schools = leaid_df['State'] == 'TX'
leaid_df = leaid_df[tx_schools]
leaid_df.head()

,LEAID,School District,State,County
15763,4800001,CROSBYTON CISD,TX,Crosby County
15764,4800002,SPUR ISD,TX,Dickens County
15765,4800003,ROCKSPRINGS ISD,TX,Edwards County
15766,4800004,POR VIDA ACADEMY,TX,Bexar County
15767,4800005,BLACKWELL CISD,TX,Nolan County


In [49]:
leaid_df.loc[:,'LEAID'] = leaid_df['LEAID'] - 4800000
leaid_df.head()

,LEAID,School District,State,County
15763,1,CROSBYTON CISD,TX,Crosby County
15764,2,SPUR ISD,TX,Dickens County
15765,3,ROCKSPRINGS ISD,TX,Edwards County
15766,4,POR VIDA ACADEMY,TX,Bexar County
15767,5,BLACKWELL CISD,TX,Nolan County


In [50]:
leaid_df = get_public_school_dis(leaid_df, 'School District')
leaid_df = remove_cisd_isd_from_col(leaid_df, 'School District')
leaid_df['School District'] = leaid_df['School District'].str.title()
leaid_df.head()

,LEAID,School District,State,County
15763,1,Crosbyton,TX,Crosby County
15764,2,Spur,TX,Dickens County
15765,3,Rocksprings,TX,Edwards County
15767,5,Blackwell,TX,Nolan County
15768,6,Aspermont,TX,Stonewall County


In [51]:
leaid_df.reset_index(drop=True, inplace=True)
leaid_df.head()

,LEAID,School District,State,County
0,1,Crosbyton,TX,Crosby County
1,2,Spur,TX,Dickens County
2,3,Rocksprings,TX,Edwards County
3,5,Blackwell,TX,Nolan County
4,6,Aspermont,TX,Stonewall County


In [52]:
sat_df_before_merge_schools = list(sat_df['School District'])
sat_df = sat_df.merge(leaid_df,left_on=('School District','County'), right_on=('School District','County'), how= 'left')
sat_df_after_merge_schools = list(sat_df['School District'])
sat_df.head()

,District Number,School District,County,2016 class SAT mean,Expenditures per Student,Economically Disadvantaged Student Count,Average Daily Attendance,Annual Dropout Rate,LEAID,State
0,1902,Cayuga,Anderson County,978.0,14278.0,210,560.125,1.0,13200.0,TX
1,1903,Elkhart,Anderson County,978.0,8509.0,648,1134.768,0.0,18390.0,TX
2,1904,Frankston,Anderson County,932.0,8994.0,446,713.236,0.3,19770.0,TX
3,1906,Neches,Anderson County,971.0,10349.0,156,396.754,1.0,32250.0,TX
4,1907,Palestine,Anderson County,918.0,10021.0,2458,3052.135,0.6,34050.0,TX


In [53]:
#find rows with missing rows
sat_df.head()
#len(set(sat_df_after_merge_schools))
#sat_df.isnull().values.any() #sum()
#sat_df[sat_df['LEAID'] == ]
sat_df[sat_df.isnull().any(axis=1)]

,District Number,School District,County,2016 class SAT mean,Expenditures per Student,Economically Disadvantaged Student Count,Average Daily Attendance,Annual Dropout Rate,LEAID,State
66,18904,Valley Mills,Bosque County,981.0,9426.0,384,573.165,0.0,NaN,NaN
112,31916,South Texas,Cameron County,1035.0,13464.0,1935,3213.220,0.0,NaN,NaN
165,50909,Jonesboro,Coryell County,933.0,9011.0,91,146.361,0.0,NaN,NaN
200,62903,Yoakum,DeWitt County,888.0,12460.0,1139,1491.059,0.1,NaN,NaN
261,84904,La Marque,Galveston County,754.0,10236.0,1592,2445.524,1.7,NaN,NaN
324,101914,Katy,Harris County,1074.0,8907.0,20057,61425.623,0.7,NaN,NaN
408,121902,Brookeland,Jasper County,1053.0,12131.0,243,376.254,0.0,NaN,NaN
656,212902,Bullard,Smith County,946.0,8083.0,866,2043.139,0.3,NaN,NaN


### After further inspection of the missing values, it was determined that they are located in multiple counties, and they are listed under different counties in the two data sets. They are however majorly located in the county listed in the sat_df from the sat score data, so I will manually add their LEAID and STATE to the sat_df where they are currently missing.

In [54]:
sat_df.fillna(value={'State':'TX'}, axis=0, inplace=True)
sat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 0 to 792
Data columns (total 10 columns):
District Number                             793 non-null int64
School District                             793 non-null object
County                                      793 non-null object
2016 class SAT mean                         793 non-null float64
Expenditures per Student                    793 non-null float64
Economically Disadvantaged Student Count    793 non-null int64
Average Daily Attendance                    793 non-null float64
Annual Dropout Rate                         793 non-null float64
LEAID                                       785 non-null float64
State                                       793 non-null object
dtypes: float64(5), int64(2), object(3)
memory usage: 68.1+ KB


In [55]:
#fills missing LEAIDs with LEAIDs from leaid_df at row with same school district name if found
# or -1 if not found
for i,row in sat_df[sat_df.isnull().any(axis=1)].iterrows():
    index_sat = sat_df.index[sat_df['School District'] == row[1]]
    index_leaid = leaid_df.index[leaid_df['School District'] == row[1]]
    if index_leaid.size == 0:
        value = -1
    else:
        value = leaid_df.iloc[index_leaid[0],0]
    sat_df.iloc[index_sat[0],8] = value

In [56]:
sat_df[sat_df['LEAID'] == -1]
sat_df.info() #same number of values in all rows now

<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 0 to 792
Data columns (total 10 columns):
District Number                             793 non-null int64
School District                             793 non-null object
County                                      793 non-null object
2016 class SAT mean                         793 non-null float64
Expenditures per Student                    793 non-null float64
Economically Disadvantaged Student Count    793 non-null int64
Average Daily Attendance                    793 non-null float64
Annual Dropout Rate                         793 non-null float64
LEAID                                       793 non-null float64
State                                       793 non-null object
dtypes: float64(5), int64(2), object(3)
memory usage: 68.1+ KB


In [57]:
sat_df.head()

,District Number,School District,County,2016 class SAT mean,Expenditures per Student,Economically Disadvantaged Student Count,Average Daily Attendance,Annual Dropout Rate,LEAID,State
0,1902,Cayuga,Anderson County,978.0,14278.0,210,560.125,1.0,13200.0,TX
1,1903,Elkhart,Anderson County,978.0,8509.0,648,1134.768,0.0,18390.0,TX
2,1904,Frankston,Anderson County,932.0,8994.0,446,713.236,0.3,19770.0,TX
3,1906,Neches,Anderson County,971.0,10349.0,156,396.754,1.0,32250.0,TX
4,1907,Palestine,Anderson County,918.0,10021.0,2458,3052.135,0.6,34050.0,TX


In [58]:
#join 2014-2015 sex and race text into sat_df
sex_race_df = pd.read_table('./CDP05_105_04000US48_105204558448.txt', header= 0, sep= "|",\
                        names=['School District','LEAID','Percent Male','Percent Female',\
                               'Percent White','Percent Black', 'Percent Native American',\
                               'Percent Asian', 'Percent Pacific Islander', 'Percent other race'],usecols = [1,2,9,33,71,75,79,99,131,151])
sex_race_df.head()

,School District,LEAID,Percent Male,Percent Female,Percent White,Percent Black,Percent Native American,Percent Asian,Percent Pacific Islander,Percent other race
0,"Abbott Independent School District, TX",7380,51.0,49.0,98.0,0.0,1.6,0.0,0.0,0.0
1,"Abernathy Independent School District, TX",7410,58.4,41.6,84.5,0.0,0.0,0.0,0.0,2.5
2,"Abilene Independent School District, TX",7440,51.6,48.3,68.4,12.7,0.4,1.6,0.0,9.6
3,"Academy Independent School District, TX",7470,71.7,28.3,89.8,3.6,0.0,1.2,0.0,2.4
4,"Agua Dulce Independent School District, TX",7530,37.3,62.7,89.8,6.8,3.4,0.0,0.0,0.0


In [59]:
#try to fill any missing (-1) LEAIDS with values from sex_race_df
for i,row in sat_df[sat_df['LEAID'] == -1].iterrows():
    index_sat = sat_df.index[sat_df['School District'] == row[1]]
    index_sex_race = sex_race_df.index[sex_race_df['School District'] == f"{row[1]} Independent School District, TX"]
    if index_sex_race.size == 0:
        value = -1
    else:
        value = sex_race_df.iloc[index_sex_race[0],1]
    sat_df.iloc[index_sat[0],8] = value
sat_df[sat_df['LEAID'] == -1] #hopefully empty!


,District Number,School District,County,2016 class SAT mean,Expenditures per Student,Economically Disadvantaged Student Count,Average Daily Attendance,Annual Dropout Rate,LEAID,State


In [60]:
sex_race_df = remove_cisd_isd_from_col(sex_race_df, 'School District')

#is_isd_cisd = re.compile(r'\s(CISD|ISD|CONSOLIDATED ISD|CONSOLIDATED CISD|Consolidated Independent School District, TX|Independent School District, TX)')
#sex_race_df['School District'] =sex_race_df['School District'].replace(to_replace=is_isd_cisd, value='') 
sex_race_df.head()

,School District,LEAID,Percent Male,Percent Female,Percent White,Percent Black,Percent Native American,Percent Asian,Percent Pacific Islander,Percent other race
0,Abbott,7380,51.0,49.0,98.0,0.0,1.6,0.0,0.0,0.0
1,Abernathy,7410,58.4,41.6,84.5,0.0,0.0,0.0,0.0,2.5
2,Abilene,7440,51.6,48.3,68.4,12.7,0.4,1.6,0.0,9.6
3,Academy,7470,71.7,28.3,89.8,3.6,0.0,1.2,0.0,2.4
4,Agua Dulce,7530,37.3,62.7,89.8,6.8,3.4,0.0,0.0,0.0


In [61]:
#drop school district from sex_race_df
sex_race_df.drop(columns= 'School District', inplace=True)
sex_race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 877 entries, 0 to 876
Data columns (total 9 columns):
LEAID                       877 non-null int64
Percent Male                877 non-null float64
Percent Female              877 non-null float64
Percent White               877 non-null float64
Percent Black               877 non-null float64
Percent Native American     877 non-null float64
Percent Asian               877 non-null float64
Percent Pacific Islander    877 non-null float64
Percent other race          877 non-null float64
dtypes: float64(8), int64(1)
memory usage: 61.8 KB


In [62]:
sex_race_df['LEAID'] = sex_race_df['LEAID'].astype(float)
sat_df = sat_df.merge(sex_race_df,left_on=('LEAID'), right_on=('LEAID'), how= 'left')
sat_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 793 entries, 0 to 792
Data columns (total 18 columns):
District Number                             793 non-null int64
School District                             793 non-null object
County                                      793 non-null object
2016 class SAT mean                         793 non-null float64
Expenditures per Student                    793 non-null float64
Economically Disadvantaged Student Count    793 non-null int64
Average Daily Attendance                    793 non-null float64
Annual Dropout Rate                         793 non-null float64
LEAID                                       793 non-null float64
State                                       793 non-null object
Percent Male                                735 non-null float64
Percent Female                              735 non-null float64
Percent White                               735 non-null float64
Percent Black                               735 non-null 

In [63]:
sat_df.head()
#sat_df['LEAID'] = sat_df['LEAID'].astype(float)
#sat_df.info()
sat_df[sat_df.isnull().any(axis=1)]

,District Number,School District,County,2016 class SAT mean,Expenditures per Student,Economically Disadvantaged Student Count,Average Daily Attendance,Annual Dropout Rate,LEAID,State,Percent Male,Percent Female,Percent White,Percent Black,Percent Native American,Percent Asian,Percent Pacific Islander,Percent other race
28,10901,Medina,Bandera County,927.0,12789.0,163,249.439,0.7,30030.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,17901,Borden County,Borden County,1020.0,19898.0,60,229.243,0.0,10860.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,18907,Kopperl,Bosque County,951.0,10663.0,164,217.402,0.0,25890.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,18908,Cranfills Gap,Bosque County,910.0,13060.0,83,115.892,0.0,15570.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,25905,May,Brown County,1183.0,9784.0,169,237.922,0.7,29520.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,28906,Prairie Lea,Caldwell County,918.0,13420.0,131,187.972,0.0,35700.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,31916,South Texas,Cameron County,1035.0,13464.0,1935,3213.220,0.0,37150.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,36903,East Chambers,Chambers County,945.0,9191.0,802,1279.425,0.0,17880.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,42905,Panther Creek,Coleman County,955.0,14229.0,97,150.654,2.9,7.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,47905,Sidney,Comanche County,800.0,10112.0,100,129.488,0.0,40170.0,TX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
#for now drop any rows with null. Should look more into these to see why theyre null or try to fix
#school col in sex_race and then merge on school district and Leaid
sat_df.dropna(axis=0, how='any', inplace= True)
sat_df.reset_index(drop=True, inplace=True)

In [65]:
sat_df.head()

,District Number,School District,County,2016 class SAT mean,Expenditures per Student,Economically Disadvantaged Student Count,Average Daily Attendance,Annual Dropout Rate,LEAID,State,Percent Male,Percent Female,Percent White,Percent Black,Percent Native American,Percent Asian,Percent Pacific Islander,Percent other race
0,1902,Cayuga,Anderson County,978.0,14278.0,210,560.125,1.0,13200.0,TX,67.3,33.6,81.4,18.6,0.0,0.0,0.0,0.0
1,1903,Elkhart,Anderson County,978.0,8509.0,648,1134.768,0.0,18390.0,TX,55.3,44.7,97.0,1.1,0.0,0.0,0.0,0.0
2,1904,Frankston,Anderson County,932.0,8994.0,446,713.236,0.3,19770.0,TX,43.9,56.1,63.5,18.2,0.0,1.4,0.0,9.5
3,1906,Neches,Anderson County,971.0,10349.0,156,396.754,1.0,32250.0,TX,65.0,35.0,82.5,17.5,0.0,0.0,0.0,0.0
4,1907,Palestine,Anderson County,918.0,10021.0,2458,3052.135,0.6,34050.0,TX,45.5,54.5,64.1,25.7,0.0,1.0,0.0,4.3


In [66]:
#join 2010-2014 parental information text into sat_df
education_df = pd.read_table('./PDP02.5_205_04000US48_1071448307.txt', header= 0, sep= "|",\
                        names=['School District','LEAID','% Less than 9th Grade','% High School No diploma',\
                               '% High School Diploma','% Some College No Degree', "% Associate's Degree",\
                               "% Bachelor's Degree", '% Graduate or Professional Degree'],usecols = [1,2,9,13,17,21,25,29,33])
education_df.head()

,School District,LEAID,% Less than 9th Grade,% High School No diploma,% High School Diploma,% Some College No Degree,% Associate's Degree,% Bachelor's Degree,% Graduate or Professional Degree
0,"Abbott Independent School District, TX",7380,1.8,0.0,29.5,20.5,20.5,22.7,6.8
1,"Abernathy Independent School District, TX",7410,4.1,15.6,34.0,21.1,2.7,10.2,12.2
2,"Abilene Independent School District, TX",7440,6.1,10.4,26.0,30.5,7.6,13.4,6.0
3,"Academy Independent School District, TX",7470,4.3,0.6,30.5,31.9,12.1,9.2,10.6
4,"Agua Dulce Independent School District, TX",7530,5.8,7.7,28.8,21.2,15.4,11.5,7.7


In [67]:
#education_df = remove_cisd_isd_from_col(education_df, 'School District')
#is_isd_cisd = re.compile(r'\s(CISD|ISD|CONSOLIDATED ISD|CONSOLIDATED CISD|Consolidated Independent School District, TX|Independent School District, TX)')
#education_df['School District'] =education_df['School District'].replace(to_replace=is_isd_cisd, value='') 

#drop school district from education_df for now
education_df.drop(columns= 'School District', inplace=True)
education_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 8 columns):
LEAID                                801 non-null int64
% Less than 9th Grade                801 non-null float64
% High School No diploma             801 non-null float64
% High School Diploma                801 non-null float64
% Some College No Degree             801 non-null float64
% Associate's Degree                 801 non-null float64
% Bachelor's Degree                  801 non-null float64
% Graduate or Professional Degree    801 non-null float64
dtypes: float64(7), int64(1)
memory usage: 50.2 KB


In [68]:
education_df['LEAID'] = education_df['LEAID'].astype(float)
sat_df = sat_df.merge(education_df,left_on=('LEAID'), right_on=('LEAID'), how= 'left')
sat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 735 entries, 0 to 734
Data columns (total 25 columns):
District Number                             735 non-null int64
School District                             735 non-null object
County                                      735 non-null object
2016 class SAT mean                         735 non-null float64
Expenditures per Student                    735 non-null float64
Economically Disadvantaged Student Count    735 non-null int64
Average Daily Attendance                    735 non-null float64
Annual Dropout Rate                         735 non-null float64
LEAID                                       735 non-null float64
State                                       735 non-null object
Percent Male                                735 non-null float64
Percent Female                              735 non-null float64
Percent White                               735 non-null float64
Percent Black                               735 non-null 

In [69]:
#for now drop any rows with null. Should look more into these to see why theyre null or try to fix
#school col in education and then merge on school district and Leaid
sat_df.dropna(axis=0, how='any', inplace= True)
sat_df.reset_index(drop=True, inplace=True)

In [70]:
#join 2010-2014 parental financial information text into sat_df
fin_df = pd.read_table('./PDP03_205_04000US48_107158415.txt', header= 0, sep= "|",\
                        names=['School District','LEAID','Median Earnings in Dollars','% Income below Poverty Level'],usecols = [1,2,171,259])
fin_df.head()

,School District,LEAID,Median Earnings in Dollars,% Income below Poverty Level
0,"Abbott Independent School District, TX",7380,52857,10.4
1,"Abernathy Independent School District, TX",7410,27536,17.3
2,"Abilene Independent School District, TX",7440,28673,18.7
3,"Academy Independent School District, TX",7470,44440,6.4
4,"Agua Dulce Independent School District, TX",7530,47857,8.6


In [71]:
#education_df = remove_cisd_isd_from_col(education_df, 'School District')
#is_isd_cisd = re.compile(r'\s(CISD|ISD|CONSOLIDATED ISD|CONSOLIDATED CISD|Consolidated Independent School District, TX|Independent School District, TX)')
#education_df['School District'] =education_df['School District'].replace(to_replace=is_isd_cisd, value='') 

fin_df.drop(columns= 'School District', inplace=True)
fin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 3 columns):
LEAID                           801 non-null int64
Median Earnings in Dollars      801 non-null int64
% Income below Poverty Level    801 non-null float64
dtypes: float64(1), int64(2)
memory usage: 18.9 KB


In [72]:
fin_df['LEAID'] = fin_df['LEAID'].astype(float)
sat_df = sat_df.merge(fin_df,left_on=('LEAID'), right_on=('LEAID'), how= 'left')
sat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 690 entries, 0 to 689
Data columns (total 27 columns):
District Number                             690 non-null int64
School District                             690 non-null object
County                                      690 non-null object
2016 class SAT mean                         690 non-null float64
Expenditures per Student                    690 non-null float64
Economically Disadvantaged Student Count    690 non-null int64
Average Daily Attendance                    690 non-null float64
Annual Dropout Rate                         690 non-null float64
LEAID                                       690 non-null float64
State                                       690 non-null object
Percent Male                                690 non-null float64
Percent Female                              690 non-null float64
Percent White                               690 non-null float64
Percent Black                               690 non-null 

In [73]:
#join 2014-2015 total student count csv into sat_df
student_df = pd.read_csv('./Enrollment Report_Statewide_Districts_Grade_2014-2015.csv', header = 4,\
                            usecols = [4,7], names=['District Number','Student Count'])
student_df.head()

,District Number,Student Count
0,57816.0,258.0
1,57816.0,243.0
2,57816.0,157.0
3,57816.0,157.0
4,57816.0,158.0


In [74]:
student_df[student_df['Student Count'] < 0] = 0


In [75]:
print(student_df.info())
student_df = student_df.groupby('District Number').agg('sum')
student_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16645 entries, 0 to 16644
Data columns (total 2 columns):
District Number    16644 non-null float64
Student Count      16644 non-null float64
dtypes: float64(2)
memory usage: 260.2 KB
None
<class 'pandas.core.frame.DataFrame'>
Float64Index: 1217 entries, 0.0 to 254902.0
Data columns (total 1 columns):
Student Count    1217 non-null float64
dtypes: float64(1)
memory usage: 19.0 KB


In [76]:
student_df.head()

,Student Count
District Number,
0.0,0.0
1902.0,576.0
1903.0,1217.0
1904.0,799.0
1906.0,370.0


In [77]:
student_df.drop(student_df.index[0], inplace=True)
student_df.reset_index(inplace=True)

In [78]:
#student_df.drop(columns=['level_0', 'index'], inplace=True)
student_df['District Number'] = student_df['District Number'].astype(int)
student_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216 entries, 0 to 1215
Data columns (total 2 columns):
District Number    1216 non-null int64
Student Count      1216 non-null float64
dtypes: float64(1), int64(1)
memory usage: 19.1 KB


In [79]:
sat_df = sat_df.merge(student_df,left_on=('District Number'), right_on=('District Number'), how= 'left')

In [80]:
sat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 690 entries, 0 to 689
Data columns (total 28 columns):
District Number                             690 non-null int64
School District                             690 non-null object
County                                      690 non-null object
2016 class SAT mean                         690 non-null float64
Expenditures per Student                    690 non-null float64
Economically Disadvantaged Student Count    690 non-null int64
Average Daily Attendance                    690 non-null float64
Annual Dropout Rate                         690 non-null float64
LEAID                                       690 non-null float64
State                                       690 non-null object
Percent Male                                690 non-null float64
Percent Female                              690 non-null float64
Percent White                               690 non-null float64
Percent Black                               690 non-null 

In [83]:
#join 2014-2015 total avg enrollment csv into sat_df
class_df = pd.read_csv('./FteEnrollStateDistrict15.csv', header = 4,\
                            usecols = [4,11], names=['District Number','Avg. Class Enrollment'])
class_df.head()

,District Number,Avg. Class Enrollment
0,1902.0,23.0
1,1902.0,8.0
2,1902.0,20.0
3,1902.0,9.0
4,1902.0,7.0


In [84]:
class_df[class_df['Avg. Class Enrollment'] < 0] = 0

In [85]:
class_df = class_df.groupby('District Number').agg('mean')
class_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1219 entries, 0.0 to 254902.0
Data columns (total 1 columns):
Avg. Class Enrollment    1219 non-null float64
dtypes: float64(1)
memory usage: 19.0 KB


In [87]:
class_df.drop(class_df.index[0], inplace=True)
class_df.reset_index(inplace=True)

In [88]:
class_df.head()

,District Number,Avg. Class Enrollment
0,1902.0,29.798077
1,1903.0,65.583333
2,1904.0,49.950000
3,1906.0,27.046512
4,1907.0,127.005128


In [89]:
class_df['District Number'] = class_df['District Number'].astype(int)
class_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
District Number          1218 non-null int64
Avg. Class Enrollment    1218 non-null float64
dtypes: float64(1), int64(1)
memory usage: 19.2 KB


In [90]:
sat_df = sat_df.merge(class_df,left_on=('District Number'), right_on=('District Number'), how= 'left')
sat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 690 entries, 0 to 689
Data columns (total 29 columns):
District Number                             690 non-null int64
School District                             690 non-null object
County                                      690 non-null object
2016 class SAT mean                         690 non-null float64
Expenditures per Student                    690 non-null float64
Economically Disadvantaged Student Count    690 non-null int64
Average Daily Attendance                    690 non-null float64
Annual Dropout Rate                         690 non-null float64
LEAID                                       690 non-null float64
State                                       690 non-null object
Percent Male                                690 non-null float64
Percent Female                              690 non-null float64
Percent White                               690 non-null float64
Percent Black                               690 non-null 

In [ ]:
sat_df.head()

In [ ]:
### MVP